In [23]:
import pickle
import mlflow
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

In [2]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2024/05/26 12:21:46 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2024/05/26 12:21:46 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='/Users/gbemidebe/Documents/GitHub/mlops-zoomcamp/02-experiment-tracking/examples/case01/mlruns/1', creation_time=1716744106270, experiment_id='1', last_update_time=1716744106270, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

execute  `mlflow ui --backend-store-uri sqlite:///mlflow.db` in terminal

In [4]:
# get data: check if directory exist. if yes, skip else create new one
import os, wget
data_directory = 'data/'
if os.path.exists(data_directory):
    print("Directory exists")
else:
    os.mkdir(data_directory)
    print("Directory does not exist and created new one")

# set directory
os.chdir(f'./{data_directory}')
# Download files
remote_desktop = False
if remote_desktop:
    if not os.path.exists('green_tripdata_2021-01.parquet'):
        !wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet
    if not os.path.exists('green_tripdata_2021-02.parquet'):
        !wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet
else:
    if not os.path.exists('green_tripdata_2021-01.parquet'):
        wget.download('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
    if not os.path.exists('green_tripdata_2021-02.parquet'):
        wget.download('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')
os.chdir(f'../')

Directory does not exist and created new one


In [5]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [6]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [7]:
len(df_train), len(df_val)

(73908, 61921)

In [8]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [9]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [10]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [11]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

/Users/gbemidebe/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.758715208946364

In [12]:
# save model: check if directory exist. if yes, skip else create new one
directory = 'saved_models'
if os.path.exists(directory):
    print("Directory exists")
else:
    os.mkdir(directory)
    print("Directory does not exist and thus, created")

Directory does not exist and thus, created


In [13]:
with open('saved_models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

# Move MLflow

## Case 1: Simple - Use artifact

In [24]:
with mlflow.start_run():

    # set tags e.g. developer name
    mlflow.set_tag("developer", "Wale")
    # log parameters
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    mlflow.log_metric("rmse", root_mean_squared_error(y_val, y_pred))
    mlflow.log_metric("r2", r2_score(y_val, y_pred))
    mlflow.log_metric("mae", mean_absolute_error(y_val, y_pred))

    mlflow.log_artifact(local_path="saved_models/lin_reg.bin", artifact_path="models_pickle")

## Case 2: Simple - Use standard model pickle

In [25]:
with mlflow.start_run():
    # set tags e.g. developer name
    mlflow.set_tag("developer", "Adelodun")
    # log parameters
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.8
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    mlflow.log_metric("rmse", root_mean_squared_error(y_val, y_pred))
    mlflow.log_metric("r2", r2_score(y_val, y_pred))
    mlflow.log_metric("mae", mean_absolute_error(y_val, y_pred))

    mlflow.sklearn.log_model(lr, "lasso_model")

/Users/gbemidebe/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/Users/gbemidebe/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## Case 2: Hyperparameter tuning

In [20]:
import xgboost as xgb

train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [26]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        model = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50, verbose_eval=False
        )
        y_pred = model.predict(valid)
        mlflow.log_metric("rmse", root_mean_squared_error(y_val, y_pred))
        mlflow.log_metric("r2", r2_score(y_val, y_pred))
        mlflow.log_metric("mae", mean_absolute_error(y_val, y_pred))

    return {
            'loss': root_mean_squared_error(y_val, y_pred), 
            'status': STATUS_OK}

In [28]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 1, 100, 10)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)

100%|██████████| 10/10 [06:22<00:00, 38.23s/trial, best loss: 6.328294926027175]


# use autolog

In [29]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)
    
    # culled from the instructor solution
    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.xgboost.autolog()

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50, verbose_eval=False
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)

    with open("saved_models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)



/Users/gbemidebe/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [13:20:38] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
2024/05/26 13:21:34 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/05/26 13:21:34 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/gbemidebe/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [13:21:34] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
2024/05/26 13:21:35 WARNING mlflow.utils.autologging_uti

# specific logging

In [35]:
mlflow.xgboost.autolog(disable=True)

In [36]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)
    
    # culled from the instructor solution
    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:squarederror',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50, verbose_eval=False
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("saved_models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact(local_path="saved_models/preprocessor.b", artifact_path="preprocessor") # save preprocessor from local_path into artifact_path

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow") # save model 'booster' into artifact_path

/Users/gbemidebe/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [13:36:31] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/Users/gbemidebe/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/Users/gbemidebe/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/_

# Load trained model

In [38]:
logged_model = 'runs:/a85101ca28ea4d2ab6b78059aff9bfdb/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [39]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: a85101ca28ea4d2ab6b78059aff9bfdb

In [41]:
loaded_model.predict(X_val)

array([14.782765 ,  7.184751 , 15.971323 , ..., 13.464008 ,  6.7352147,
        8.184539 ], dtype=float32)